In [1]:
import nltk
import pandas as pd
import requests
import seaborn as sns
import shutil
from IPython.display import display

from pathlib import Path
from typing import Optional

In [2]:
sns.set_style("whitegrid")
sns.set_context("talk")

In [3]:
def download_file(url:str, local_filename:Optional[str]=None):
    if local_filename is None:
      local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return local_filename

def unzip_file(path_to_zip_file:str, directory_to_extract_to:str):
  shutil.unpack_archive(path_to_zip_file, directory_to_extract_to)

In [4]:
# Check if the data already exists, if not, download and unzip it
data_path = Path('data')
if not (data_path / 'laboratorios.csv').is_file() and not (data_path / 'notas.csv').is_file() and not (data_path / 'sociodemografico.csv').is_file():
  data_zip = download_file('https://www.dropbox.com/sh/xgs3kyvyn7lmr6p/AACB4eORnqsJpRsjv9-56eUHa?dl=1', 'data.zip')
  unzip_file(data_zip, data_path)
  unzip_file(data_path / 'Diana Buitrago - IQVIA_NLPmediaclNotes_DianaBuitrago.zip', data_path)

In [5]:
# Check if the data already exists, if not, download and unzip it
data_path = Path('data')
if not (data_path / 'laboratorios.csv').is_file() and not (data_path / 'notas.csv').is_file() and not (data_path / 'sociodemografico.csv').is_file():
  data_zip = download_file('https://www.dropbox.com/sh/xgs3kyvyn7lmr6p/AACB4eORnqsJpRsjv9-56eUHa?dl=1', 'data/data.zip')
  unzip_file(data_zip, data_path)
  unzip_file(data_path / 'Diana Buitrago - IQVIA_NLPmediaclNotes_DianaBuitrago.zip', data_path)

In [6]:
notas = pd.read_csv(str(data_path / "notas.csv"),sep=';')
notas.head()

,IDRecord,Código,Nombre,Tipo,Plan
0,44600,A539,"SIFILIS, NO ESPECIFICADA",Confirmado Repetido,- ORDENO TAR ABC +3TC +ATV/r - PROFILAXIS NO ...
1,45038,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,- TAF/FTC/EVG/C MIPRES POR 2 MESES 20200602158...
2,40391,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,usuaria la cual se ve pertinente seguimiento d...
3,106350,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,1. Se formula TAR (TDF/FTC+EFV) 2. S/S Paracl...
4,105840,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,EDUCACIÓN Se brinda retroalimentación con rel...


Cleanup based on initial EDA

In [7]:
from utils.preprocessing_utils import preprocess_notas

notas = preprocess_notas(notas)
notas.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,IDRecord,Código,Nombre,Tipo,Plan
0,44600.0,A539,"SIFILIS, NO ESPECIFICADA",Confirmado Repetido,- ORDENO TAR ABC +3TC +ATV/r - PROFILAXIS - F...
1,45038.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,- TAF/FTC/EVG/C MIPRES 2 MESES 202006021580194...
2,40391.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,usuaria ve pertinente seguimiento dentro mes m...
3,106350.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,1. formula TAR (TDF/FTC+EFV) 2. S/S Paraclini...
4,105840.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,EDUCACION brinda retroalimentacion relacion r...


Let's check our taget feature distribution

In [8]:
name = notas[['Nombre', 'Código']].value_counts().to_frame('Count')
name['Percentage'] = (name.Count / sum(name.Count) * 100).round(2)
name

,,Count,Percentage
Nombre,Código,,
"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O TARDIA",A530,60587,43.22
"SIFILIS, NO ESPECIFICADA",A539,47408,33.82
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MENCION DE COMPLICACION,E119,17439,12.44
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCION DE COMPLICACION,E109,6278,4.48
"DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION DE COMPLICACION",E149,2808,2.00
OTRAS SIFILIS SECUNDARIAS,A514,2614,1.86
"SIFILIS TARDIA, NO ESPECIFICADA",A529,1970,1.41
SIFILIS GENITAL PRIMARIA,A510,977,0.70
SIFILIS PRIMARIA ANAL,A511,94,0.07


After the data cleanup, we only have less than 1% for two of the 9 classes, representing a total of ~1000 samples of the ~150,000 in our dataset.
We can't easily create a prediction algorithm out of these small number of samples, so let's either drop them or merge them into similar categories.

## Classes merge

Let's try merging A510 and A511 with A514, as they all belong to the [A51 Early syphilis](https://icd.who.int/browse10/2019/en#/A51) ICD-10 denomination, indicating they share symptoms. The equivalent Spanish name for this category is [Sífilis precoz](http://ais.paho.org/classifications/chapters/CAP01.html?zoom_highlight=a51).

We could also treat A529 as part of A539, in order to reduce the number of classes without losing track of the important subcategories we already have.

In [9]:
from utils.preprocessing_utils import merge_classes

notas = merge_classes(notas)
name = notas[['Nombre', 'Código']].value_counts().to_frame('Count')
name['Percentage'] = (name.Count / sum(name.Count) * 100).round(2)
name

,,Count,Percentage
Nombre,Código,,
"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O TARDIA",A530,60587,43.22
"SIFILIS, NO ESPECIFICADA",A539,49378,35.23
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MENCION DE COMPLICACION,E119,17439,12.44
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCION DE COMPLICACION,E109,6278,4.48
SIFILIS PRECOZ,A51,3685,2.63
"DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION DE COMPLICACION",E149,2808,2.00


## Word count

According to the [CDC](https://www.cdc.gov/std/syphilis/stdfact-syphilis-detailed.htm), primary syphilis is characterized by a chancre mark where the disease enters the body. There is also a possibility of having extra sores in your body, but there does not seem to be any difference per se in the development of its condition based on where the disease started. Another clear indication of syphilis is Saber shin (pierna/tibia en sable).
There is also a reduction in cognitive abilities for patients who have been suffering of syphilis for some time, and this can be tested for using a simple test called the [Clock Drawing Test](https://www2.gov.bc.ca/assets/gov/health/practitioner-pro/bc-guidelines/cogimp-clock-drawing-test.pdf), or [Test del Reloj](https://www.sanitas.es/sanitas/seguros/es/particulares/biblioteca-de-salud/tercera-edad/demencias/test-reloj.html) in Spanish.

Additionally, a main characteristic of primary syphilis seems to be chancres, as well as sores for both Primary and Secondary Syphilis, making a case for creating a new numerical variable called "chancres". Another main characteristic of Syphilis is the push to use preservatives in order to reduce the possibility of other people being infected as well, which could help differentiate between Syphilis and Diabetes.

Other related keywords we can try is genital, skin (lesions), headache, HIV, serology, hepatitis, and specific tests performed on the patients.

In [13]:

notas_eda = notas.copy()
words_to_check = [
    'chancro',
    'llaga',
    'preservativo',
    'sifili',
    'asintoma',
    'placa',
    r'(test.*reloj)',
    'sable',
    'penici',
    'antibio',
    'genital',
    'piel',
    'lesion',
    'macula',
    'cabeza',
    'vih',
    'FTA-ABS',
    'serolo',
    'hepatitis',
    'VDRL',
    'RPR',
    ]
aggregate_dict = {}
for word in words_to_check:
    notas_eda[word] = notas_eda.Plan.str.lower().str.count(word.lower())
    notas_eda.loc[notas_eda[word] > 1, word] = 1
    aggregate_dict[word] = ['sum']
aggregate_dict['Nombre'] = ['count']
notas_eda = notas_eda.groupby(['Nombre', 'Código'])[words_to_check + ['Nombre']].aggregate(aggregate_dict)

for word in words_to_check:
    notas_eda.loc(axis=1)[word, '%'] = (notas_eda.loc(axis=1)[word, 'sum'] / notas_eda.loc(axis=1)['Nombre', 'count'] * 100).round(2)
with pd.option_context('display.max_columns', None):
    display(notas_eda.sort_index(axis=1).sort_values(by=['Código']))

(test.*reloj)        \
                                                                      %   sum   
Nombre                                             Código                       
SIFILIS PRECOZ                                     A51             4.91   181   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530            2.91  1761   
SIFILIS, NO ESPECIFICADA                           A539            1.71   845   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109            1.47    92   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119            1.32   230   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149            1.28    36   

                                                          FTA-ABS     Nombre  \
                                                                % sum  count   
Nombre                                             Código                      
SIFILIS PRECOZ                                     A51       0.00   0   3685   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530      0.02  11  60587   
SIFILIS, NO ESPECIFICADA                           A539      0.06  29  49378   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109      0.02   1   6278   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119      0.00   0  17439   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149      0.00   0   2808   

                                                             RPR        VDRL  \
                                                               %   sum     %   
Nombre                                             Código                      
SIFILIS PRECOZ                                     A51      8.06   297  0.54   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530     8.74  5295  0.78   
SIFILIS, NO ESPECIFICADA                           A539    18.17  8971  0.67   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109     4.64   291  0.33   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119     2.92   510  0.23   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149     3.95   111  0.28   

                                                               antibio       \
                                                           sum       %  sum   
Nombre                                             Código                     
SIFILIS PRECOZ                                     A51      20    0.11    4   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530    475    0.10   58   
SIFILIS, NO ESPECIFICADA                           A539    329    0.06   28   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109     21    2.80  176   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119     40    2.99  522   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149      8    5.38  151   

                                                          asintoma        \
                                                                 %   sum   
Nombre                                             Código                  
SIFILIS PRECOZ                                     A51        5.37   198   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530       3.07  1861   
SIFILIS, NO ESPECIFICADA                           A539       1.51   748   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109       1.85   116   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119       2.44   426   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149       2.35    66   

                                                          cabeza         \
                                                               %    sum   
Nombre                                             Código                 
SIFILIS PRECOZ                                     A51     26.08    961   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530    18.07  10950   
SIFILIS, NO ESPECIFICADA                           A539     9.17   4529   
DIABETES MELLITUSINSULINODEPENDIENTE SI

For diabetes, we tried adding insulin and glucose as words of interest. Ketoacidosis is another relevant word which we can separate into keto and acido to see if we can capture more information.

Other keywords associated with diabetes are: obesity, carbohydrates, overweight, polyphagia, polydipsia, polyurea

In [11]:
notas_eda = notas.copy()
words_to_check = [
    'ampolla',
    'diabet',
    'insulin',
    'gluco',
    'carbo',
    'keto',
    'acido',
    'nutri',
    'diet',
    'dependiente',
    'obes',
    'sobrepeso',
    'polifagia',
    'polidipsia',
    'poliurea',
    ]
aggregate_dict = {}
for word in words_to_check:
    notas_eda[word] = notas_eda.Plan.str.lower().str.count(word.lower())
    notas_eda.loc[notas_eda[word] > 1, word] = 1
    aggregate_dict[word] = ['sum']
aggregate_dict['Nombre'] = ['count']
notas_eda = notas_eda.groupby(['Nombre', 'Código'])[words_to_check + ['Nombre']].aggregate(aggregate_dict)

for word in words_to_check:
    notas_eda.loc(axis=1)[word, '%'] = (notas_eda.loc(axis=1)[word, 'sum'] / notas_eda.loc(axis=1)['Nombre', 'count'] * 100).round(2)
notas_eda.sort_index(axis=1).sort_values(by=['Código'])
with pd.option_context('display.max_columns', None):
    display(notas_eda.sort_index(axis=1).sort_values(by=['Código']))

Nombre acido        \
                                                           count     %   sum   
Nombre                                             Código                      
SIFILIS PRECOZ                                     A51      3685  0.41    15   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530    60587  0.92   558   
SIFILIS, NO ESPECIFICADA                           A539    49378  1.70   838   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109     6278  6.90   433   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119    17439  6.46  1126   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149     2808  5.59   157   

                                                          ampolla      carbo  \
                                                                %  sum     %   
Nombre                                             Código                      
SIFILIS PRECOZ                                     A51       0.24    9  3.88   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530      0.38  233  3.26   
SIFILIS, NO ESPECIFICADA                           A539      0.95  471  2.22   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109      2.10  132  3.17   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119      1.29  225  2.07   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149      1.07   30  2.60   

                                                                dependiente  \
                                                            sum           %   
Nombre                                             Código                     
SIFILIS PRECOZ                                     A51      143        1.22   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530    1973        1.44   
SIFILIS, NO ESPECIFICADA                           A539    1094        0.74   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109     199        0.67   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119     361        1.23   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149      73        0.68   

                                                               diabet       \
                                                           sum      %  sum   
Nombre                                             Código                    
SIFILIS PRECOZ                                     A51      45   0.14    5   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530    875   0.26  160   
SIFILIS, NO ESPECIFICADA                           A539    365   0.28  136   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109     42   5.16  324   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119    214   5.55  968   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149     19   6.16  173   

                                                            diet        gluco  \
                                                               %   sum      %   
Nombre                                             Código                       
SIFILIS PRECOZ                                     A51      7.54   278   1.03   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530     8.38  5078   1.08   
SIFILIS, NO ESPECIFICADA                           A539     7.71  3809   1.04   
DIABETES MELLITUSINSULINODEPENDIENTE SIN MENCIO... E109    13.35   838  12.68   
DIABETES MELLITUS NOINSULINODEPENDIENTE SIN MEN... E119    11.72  2044   5.84   
DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION ... E149    13.35   375   9.44   

                                                                insulin        \
                                                            sum       %   sum   
Nombre                                             Código                       
SIFILIS PRECOZ                                     A51       38    0.03     1   
SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O ... A530     652    0.09    56   
SIFILIS, NO ESPECIFICADA                           A539     512   

- There does not seem to be many mentions of chancre (chancro) or sore (llaga/placa)
- Ulcer (ulcera) does seem to be more common in patients with diabetes, although it still is negligible.
- Saber (sable) seems to be able to help differentiate for all but 1 types of syphilis against diabetes.
- asintoma seems to be useful for separating other secondary syphilis from the rest of the diseases.
- There does seem to be a significant difference between the times the word preservative (preservativo) is used between Syphilis and Diabetes.
- insulin seems like a good choice for separating E109 from the rest.
- acido, keto and diet seem to also help differentiate between syphilis and diabetes.
- Using 'diabet' and 'sifili' could help differentiate between the diagnoses of Diabetes and Syphilis.

In [12]:
from utils.preprocessing_utils import word_count_feat_engineering

notas = word_count_feat_engineering(notas)
with pd.option_context('display.max_columns', None):
    display(notas.head(10))

,IDRecord,Código,Nombre,Tipo,Plan,acido,antibio,asintoma,diabet,diet,gluco,insulin,keto,penici,preservativo,sable,sifili,test_reloj_orden
0,44600.0,A539,"SIFILIS, NO ESPECIFICADA",Confirmado Repetido,- ORDENO TAR ABC +3TC +ATV/r - PROFILAXIS - F...,0,0,0,0,0,0,0,0,0,1,0,0,0
1,45038.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,- TAF/FTC/EVG/C MIPRES 2 MESES 202006021580194...,0,0,0,0,0,0,0,0,0,0,0,0,0
2,40391.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,usuaria ve pertinente seguimiento dentro mes m...,0,0,0,0,0,0,0,0,0,0,0,0,0
3,106350.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,1. formula TAR (TDF/FTC+EFV) 2. S/S Paraclini...,0,0,0,0,0,0,0,0,0,0,0,0,0
4,105840.0,A530,"SIFILIS LATENTE, NO ESPECIFICADA COMO PRECOZ O...",Confirmado Repetido,EDUCACION brinda retroalimentacion relacion r...,0,0,0,0,0,0,0,0,0,1,0,0,0
5,188030.0,A539,"SIFILIS, NO ESPECIFICADA",Confirmado Repetido,1. Continuar TAR : ABC/3TC+EFV. 2. Tomar TAR ...,0,0,0,0,0,0,0,0,0,2,0,0,0
6,41590.0,A539,"SIFILIS, NO ESPECIFICADA",Confirmado Repetido,) continua manejo iniciado 19/02/2020 ABC/3TC/...,0,0,0,0,0,0,0,0,0,1,0,0,0
7,111487.0,A539,"SIFILIS, NO ESPECIFICADA",Confirmado Repetido,Control segun frecuencias,0,0,0,0,0,0,0,0,0,0,0,0,0
8,69603.0,E119,DIABETES MELLITUS NOINSULINODEPENDIENTE SIN ME...,Confirmado Repetido,realiza control enfermeria 16 OCTUBRE 2020. As...,0,0,0,0,0,0,0,0,0,2,0,0,0
9,69563.0,A51,SIFILIS PRECOZ,Confirmado Repetido,1. TRATAMIENTO TDF/FTC/EFV 2. PROFILAXIS:3....,0,0,0,0,0,0,0,0,0,2,1,0,0
